1. 小区间质押是否可能获得高额利润

In [2]:
# 获取并存储天级数据
POOL_ID = "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443"

arbitrum_query = """ query get_pools($num_skip: Int, $pool_id: ID!) {
    liquidityPool(id: $pool_id) {
        dailySnapshots(skip: $num_skip, orderBy: day, orderDirection: desc) {
            day
            dailyVolumeUSD
            dailyTotalRevenueUSD
            totalValueLockedUSD
        }
    }
}"""

In [ ]:
num_skip = 0
result = []
is_break = False

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

while True:
    print("Querying ticks, num_skip={}".format(num_skip))
    variables = {"num_skip": num_skip, "pool_id": POOL_ID}
    response = client.execute(gql(arbitrum_query), variable_values=variables)
    query_result = response["liquidityPool"]["dailySnapshots"]
    for item in query_result:
        day = int(item["day"])
        # todo: 修改这个部分
        if day <= 19477:
            is_break = True
            break
        result.append(item)

    if is_break == True or len(query_result) == 0:
        break

    num_skip = len(result)

In [4]:
import pandas as pd

from datetime import datetime, timedelta


def unixtime_to_day_before(day_count, unixtime=0):
    current_date = datetime.fromtimestamp(unixtime) + timedelta(days=day_count)
    return current_date.strftime("%Y-%m-%d")


arb_df = pd.DataFrame(result)
arb_df["date"] = arb_df["day"].apply(unixtime_to_day_before)
arb_df.head()

,day,dailyVolumeUSD,dailyTotalRevenueUSD,totalValueLockedUSD,date
0,19556,62645858.87339231038231371303131,31322.92943669615519115685651945,43991968.86397924299999433932886,2023-07-18
1,19555,78836539.34931248039460595756886,39418.2696746562401973029787891,44594966.108651452749034656584,2023-07-17
2,19554,43233649.01354901681799617409771,21616.82450677450840899808705174,46212787.3971710232147209161641,2023-07-16
3,19553,29838778.31630346908133923990871,14919.38915815173454066961995703,46504482.69780273868477842499168,2023-07-15
4,19552,129294121.16599334024223784584379,64647.06058299667012111892292883,46430790.59592811944603207193834,2023-07-14


In [6]:
arb_df["dailyVolumeUSD"] = arb_df["dailyVolumeUSD"].astype(float).round(2)
arb_df["dailyTotalRevenueUSD"] = arb_df["dailyTotalRevenueUSD"].astype(float).round(2)
arb_df["totalValueLockedUSD"] = arb_df["totalValueLockedUSD"].astype(float).round(2)
arb_df["date"] = arb_df["date"].astype(str)
arb_df = arb_df[['date', 'dailyVolumeUSD', 'dailyTotalRevenueUSD', 'totalValueLockedUSD']]

arb_old_df = pd.read_csv("../../data/arb_data_20230718.csv")
# arb_old_df.rename(columns={'dailyTotalVolumeUSD': 'dailyVolumeUSD'}, inplace=True)
arb_df = pd.concat([arb_df, arb_old_df])
arb_df.to_csv("../../data/arb_data_20230718.csv", index=False)

,date,dailyTotalVolumeUSD,dailyTotalRevenueUSD,totalValueLockedUSD
0,2023-04-30,1.294347e+08,64717.36,58969074.13
1,2023-04-29,5.831820e+07,29159.10,59522967.00
2,2023-04-28,1.291540e+08,64576.98,58623166.41
3,2023-04-27,2.670841e+08,133542.06,53907496.47
4,2023-04-26,2.884190e+08,144209.50,50551198.16


In [21]:
# 获取并存储小时级数据
POOL_ID = "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443"

arbitrum_query = """ query get_pools($num_skip: Int, $pool_id: ID!) {
    liquidityPool(id: $pool_id) {
        hourlySnapshots(skip: $num_skip, orderBy: hour, orderDirection: desc) {
            hour
            hourlyVolumeUSD
            hourlyTotalRevenueUSD
            totalValueLockedUSD
        }
    }
}"""

num_skip = 0
result = []
is_break = False

In [24]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

while True:
    print("Querying ticks, num_skip={}".format(num_skip))
    variables = {"num_skip": num_skip, "pool_id": POOL_ID}
    response = client.execute(gql(arbitrum_query), variable_values=variables)
    query_result = response["liquidityPool"]["hourlySnapshots"]
    for item in query_result:
        hour = int(item["hour"])
        # todo: 修改这个部分
        if hour <= 467775:
            is_break = True
            break
        result.append(item)

    if is_break == True or len(query_result) == 0:
        break

    num_skip = len(result)

Querying ticks, num_skip=0
Querying ticks, num_skip=100
Querying ticks, num_skip=200
Querying ticks, num_skip=300
Querying ticks, num_skip=400
Querying ticks, num_skip=500
Querying ticks, num_skip=600
Querying ticks, num_skip=700
Querying ticks, num_skip=800
Querying ticks, num_skip=900
Querying ticks, num_skip=1000
Querying ticks, num_skip=1100
Querying ticks, num_skip=1200
Querying ticks, num_skip=1300
Querying ticks, num_skip=1400
Querying ticks, num_skip=1500
Querying ticks, num_skip=1600


In [27]:
import time

def hour_to_unixtime(hour):
    current_date = datetime.fromtimestamp(0) + timedelta(hours=hour)
    return int(time.mktime(current_date.timetuple()))

In [29]:
arb_hour_df = pd.DataFrame(result)
arb_hour_df["timestamp"] = arb_hour_df["hour"].apply(hour_to_unixtime)
arb_hour_df.head()

,hour,hourlyVolumeUSD,hourlyTotalRevenueUSD,totalValueLockedUSD,timestamp
0,469376,1460878.47258988336766531052831,730.43923629494168383265526443,44233373.3510221773573453678896,1689753600
1,469375,806643.10919596517901233525319,403.3215545979825895061676268,44290156.20582897883020136578084,1689750000
2,469374,1036466.04480128782220493231525,518.23302240064391110246615779,44254886.55727854726053867876903,1689746400
3,469373,656345.72649494087873370659467,328.17286324747043936685329741,44168146.44106156636941494461425,1689742800
4,469372,1637624.49925289785898467064351,818.8122496264489294923353219,44087659.78889045440728093729744,1689739200


In [31]:
arb_hour_df[['timestamp', 'hourlyVolumeUSD', 'hourlyTotalRevenueUSD', 'totalValueLockedUSD']].to_csv("../../data/arb_hour_data_20230719.csv", index=False)

In [1]:
# 比较小时级数据回测质押收益 => 先粗略计算，再根据质押的分布精确计算

import pandas as pd

price_df = pd.read_csv("../../data/eth_usd_polygon_20230718.csv")
price_df.head()

,v,vw,o,c,h,l,t,n
0,9809.273476,1942.7081,1940.49,1952.54,1958.700000,1928.00,1654041600000,30746
1,6089.567718,1946.3557,1954.20,1946.57,1954.200000,1940.00,1654045200000,23571
2,8278.776988,1935.8301,1946.46,1939.03,1947.349248,1928.98,1654048800000,25013
3,7490.516503,1931.0554,1938.94,1928.44,1940.243055,1925.36,1654052400000,21634
4,10282.469141,1924.8137,1928.38,1932.54,1935.900000,1912.03,1654056000000,25834
